In [1]:
from utils import *
import glob
import pandas as pd
import json

## Set the path to load data.

In [2]:
# This path for loading raw data from the VISTEC nested ner corpus.
path_corpus          = '/home/module/data/vistec_corpus/data_raw/'
Format_csv           = '**/*.csv'
Format_json          = '**/*.json'

filenames_csv        = [f for f in glob.glob(path_corpus+Format_csv, recursive=True)]
filename_json        = [f for f in glob.glob(path_corpus+Format_json, recursive=True)]

# Show all files to download
print('CSV files')
[print(f) for f in filenames_csv]

print('\nJson files')
[print(f) for f in filename_json]
pass

CSV files
/home/module/data/vistec_corpus/data_raw/dataLot1/pct_wn_first_lot.csv
/home/module/data/vistec_corpus/data_raw/datalot2/pct_600review_195530.csv
/home/module/data/vistec_corpus/data_raw/datalot3/pct_600review_198387.csv

Json files
/home/module/data/vistec_corpus/data_raw/dataLot1/ner_lot1_180520_num_word_192449_num_tag_21688.json
/home/module/data/vistec_corpus/data_raw/datalot2/ner_lot2_180520_num_word_195530_num_tag_32232.json
/home/module/data/vistec_corpus/data_raw/datalot3/ner_lot3_180520_num_word_198387_num_tag_31329.json


## Set the path to save data.

In [3]:
max_levels = 8

# # This setting for subtags nested ner.
path_save            = '/home/module/data/vistec_corpus/nested_ner/subtags/'
path_save_topl       = '/home/module/data/vistec_corpus/flatten_ner/subtags/'
group                = False

# # This setting for main tags nested ner.
# path_save            = '/home/module/data/vistec_corpus/nested_ner/maintags/'
# path_save_topl       = '/home/module/data/vistec_corpus/flatten_ner/maintags/'
# group                = True

## Tags and save nested entities.

In [ ]:
%%time

temp_sentences   = []
for filen_csv, filen_json in zip(filenames_csv, filename_json):
    
    # Check the file name to load and save.
    checklotdir_csv   = filen_csv.split('/')[-2]
    checklotdir_json  = filen_json.split('/')[-2]
    save_file_name    = filen_json.split('/')[-2]
    
    assert checklotdir_csv == checklotdir_json
    print('\n',checklotdir_csv, checklotdir_json)
    print('{}'.format(filen_csv.split('/')[-1]))
    print('{}'.format( filen_json.split('/')[-1]))
    
    ## Create output files
    sftags_nested_ner = open(path_save+save_file_name+'_nested.data', 'w')

    ## Load CSV file
    data_csv  = pd.read_csv(filen_csv)
    
    ## Load Json file 
    read_file = open(filen_json, "r")
    data_json = json.load(read_file)
    
    ## Tags entities
    for idx, datas in enumerate(zip(data_json, data_csv['ssg'], data_csv['text_clean'])):
        
        ## Remove pipe
        datas = remove_pipe(datas[0],datas[1],datas[2])
        
        # Prepare data
        entities            = datas['entities']
        text                = datas['text']
        token_idx_en        = get_all_entities_index(entities)
        words_token         = token_en2words(token_idx_en, text)
        words_tags_nested   = tags_nested(words_token, entities, max_levels, group=group)
        words_tags_nested   = tags_bioes(words_tags_nested, max_levels)

#         ## Save nested corpus ##
        save_data(words_tags_nested, sftags_nested_ner, nested=True)
        temp_sentences.append(words_tags_nested)

sftags_nested_ner.close()
# check_sentence(datas)


 dataLot1 dataLot1
pct_wn_first_lot.csv
ner_lot1_180520_num_word_192449_num_tag_21688.json

 datalot2 datalot2
pct_600review_195530.csv
ner_lot2_180520_num_word_195530_num_tag_32232.json

 datalot3 datalot3
pct_600review_198387.csv
ner_lot3_180520_num_word_198387_num_tag_31329.json


In [ ]:
len(temp_sentences)

## Save top-level

In [ ]:
srcf = open(path_save_topl+'data.src', 'w')
trgf = open(path_save_topl+'data.trg', 'w')

for sent in temp_sentences:
    
    source = [token[0] for token in sent]
    target = [token[1] for token in sent]
    
    srcf.writelines('|'.join(source)+'\n')
    trgf.writelines('|'.join(target)+'\n')
    
srcf.close()
trgf.close()

## Check by sentence number.

In [ ]:
idx_sent = 0
ck_sent  = temp_sentences[idx_sent-1]
for idx, sent in enumerate(ck_sent):
    print("{:<5}".format(idx), end='')
    for token in sent:
        [print('{:<15}\t'.format(t), end='|') for t in token.split('|')]
    print()       